<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/univariate_parametric_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%capture
!git clone https://github.com/Krankile/npmf.git
!pip install wandb

In [ ]:
%%capture
!cd npmf && git pull

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##General setup

In [ ]:
%%capture
!pip install pmdarima

In [ ]:
%%capture
!cd npmf && git pull

import os
from collections import defaultdict
from collections import Counter
from datetime import datetime
from operator import itemgetter

import numpy as np
from numpy.ma.core import outerproduct
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

import wandb as wb

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import pmdarima as pm

from npmf.utils.colors import main, main2, main3
from npmf.utils.wandb import get_df_artifact
from npmf.utils.wandb import put_dataset
from npmf.utils.eikon import column_mapping
from npmf.utils.data import test_start_pd

In [ ]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
np.random.seed(420)

# Get data from WandB 😂✨KAWAIII ^^✨ 

In [ ]:
data = get_df_artifact("stock-data-clean-k-5:v0", project="master-test").rename(columns=column_mapping)
data

wandb: Currently logged in as: krankile (use `wandb login --relogin` to force relogin)


wandb: Downloading large artifact stock-data-clean-k-5:v0, 135.04MB. 1 files... Done. 0:0:0


,ticker,date,market_cap,close_price,currency
0,SZR.WA,2010-02-19,4316546.76259,0.061665,USD
1,SZR.WA,2010-02-22,6255155.347814,0.089359,USD
2,SZR.WA,2010-02-23,7567823.237272,0.108112,USD
3,SZR.WA,2010-02-24,7851000.917649,0.112157,USD
4,SZR.WA,2010-02-25,8833862.00075,0.126198,USD
...,...,...,...,...,...
5747788,300772.SZ,2022-04-15,1421837745.71855,2.621458,USD
5747789,300772.SZ,2022-04-18,1454200172.46267,2.681125,USD
5747790,300772.SZ,2022-04-19,1438043708.32942,2.651337,USD
5747791,300772.SZ,2022-04-20,1377339011.94927,2.539415,USD


### Create a split of data for training and testing

All data from, and including, January 1st, 2019 will be reserved for the test set. All other data can be used for training.

In [ ]:
train_test_split = data.date < test_start_pd

train = data[train_test_split]
test = data[~train_test_split]

train.shape, test.shape

((4484814, 5), (1262979, 5))

After the split, the training set contains ~4.5M stock prices, while the test set has ~1.3M. There are ~5 trading days in a week, ~20 in a month, ~60 in a quarter, and ~250 in a year. If forecasting a month into the future, it is reasonable to require at least a quarter's worth of training data, minimum. Therefore, we require included companies to have at least a quarter plus a month worth of days (i.e., 80) in the training set to be included.

In [ ]:
counts = train.groupby("ticker").count()

exclude_tickers = counts[counts.market_cap < 80]
exclude_tickers.index.unique().shape[0], exclude_tickers.date.sum(axis=0), exclude_tickers.date.sum(axis=0) / train.shape[0] * 1000

(16, 668, 0.14894709122830957)

When imposing this requirement, we lose 16 tickers, and ~700 datapoints, or 0.15‰ of the total training data.

In [ ]:
train = train[~train.ticker.isin(exclude_tickers.index)]
test = test[~test.ticker.isin(exclude_tickers.index)]

train.shape, test.shape

((4484146, 5), (1250576, 5))

### Combine into one frame and upload to WandB

In [ ]:
all_df = pd.concat([train, test], axis=0).sort_values(by=["ticker", "date"]).reset_index().drop(columns=["index"])

put_dataset(all_df, "stock-data-sufficiency-cleaned.feather", project="master-test")

In [ ]:
all_df.ticker.unique().shape[0], all_df.shape[0]

(1577, 5734722)